<a href="https://colab.research.google.com/github/monika-the-unicorn/Choice-assay/blob/main/Copy_of_2_Extraction_of_data_from_csvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook uses DLC to analyse a folder of new videos in the VideoDrive from a round of beecam data collection

### Does not require GPUs



*   Mounts drive and lists all csv files in the specified directory
*   Identifies which of these have already been screened for proboscis activity and had these frames extracted into output file
*   From each unprocessed csv, extracts frames of video in which parts of the proboscis were present and labels the behaviour type





# Setup

## 1. Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Specify folder to search for csvs

In [ ]:
FOLDER_NAME = "Other drugs November 2024" #ACTION REQUIRED: DEFINE DIRECTORY CONTAINING EXTRACTED DATA (folder dedicated to entire experiment)
directory_to_analyse_path = '/content/drive/My Drive/VideoDrive/' + FOLDER_NAME

In [ ]:
import os
import pandas as pd
from datetime import datetime

## 3. Specify output dataset to write labelled feeding frames to (may or may not yet exist)

In [ ]:
output_directory = "/content/drive/My Drive/BeecamData/Other drugs/DLC_output"
file_substring = "proboscis_data_Oct-25"
PATH_EXPORT = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/' + file_substring + '.csv'
print(PATH_EXPORT)

/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data_Oct-25.csv


## 4. List and count all csvs in the target directory

In [ ]:
def list_all_csvs(source_dir):
    # List to hold the paths of mp4 files
    all_csvs = []
    csv_counter = 0

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        # Iterate over all files in the current root
        for file in files:
            if file.endswith('.csv'):
              if file not in all_csvs:
                csv_path = os.path.join(root, file)
                csv_counter += 1
                all_csvs.append(csv_path)
                print(csv_counter, "csvs counted")

    print(len(all_csvs), "csvs in total in this experiment")
    return all_csvs

all_csvs = list_all_csvs(directory_to_analyse_path)

Streaming output truncated to the last 5000 lines.
82644 csvs counted
82645 csvs counted
82646 csvs counted
82647 csvs counted
82648 csvs counted
82649 csvs counted
82650 csvs counted
82651 csvs counted
82652 csvs counted
82653 csvs counted
82654 csvs counted
82655 csvs counted
82656 csvs counted
82657 csvs counted
82658 csvs counted
82659 csvs counted
82660 csvs counted
82661 csvs counted
82662 csvs counted
82663 csvs counted
82664 csvs counted
82665 csvs counted
82666 csvs counted
82667 csvs counted
82668 csvs counted
82669 csvs counted
82670 csvs counted
82671 csvs counted
82672 csvs counted
82673 csvs counted
82674 csvs counted
82675 csvs counted
82676 csvs counted
82677 csvs counted
82678 csvs counted
82679 csvs counted
82680 csvs counted
82681 csvs counted
82682 csvs counted
82683 csvs counted
82684 csvs counted
82685 csvs counted
82686 csvs counted
82687 csvs counted
82688 csvs counted
82689 csvs counted
82690 csvs counted
82691 csvs counted
82692 csvs counted
82693 csvs counted

## 5. Check if the dataset exists

*   If so, read it in (and later identify which of those csvs have already been processed)
*   If not, create the file




In [ ]:
# LOCATE THE EXISTING DATASET AND READ IN
target_file = None
# Check if any file in the directory contains the substring in its name
for file_name in os.listdir(output_directory):
    if file_substring in file_name:
        target_file = os.path.join(output_directory, file_name)
        break
if target_file:
    print(f"File found: {target_file}")
    print("Reading in dataframe")
    data = pd.read_csv(target_file)
    print("File successfully read.")
else:
    print("File not found. Creating a new empty file...")
    # Create an empty DataFrame (or specify column names if you want)
    data = pd.DataFrame(columns = ["Frame", "L_antenna_x",
    "L_antenna_y", "L_antenna_likelihood",
    "R_antenna_x", "R_antenna_y", "R_antenna_likelihood",
    "L_mandible_x", "L_mandible_y", "L_mandible_likelihood",
    "R_mandible_x", "R_mandible_y", "R_mandible_likelihood",
    "Top_prob_x", "Top_prob_y", "Top_prob_likelihood",
    "Tube_prob_x", "Tube_prob_y", "Tube_prob_likelihood",
    "End_prob_x", "End_prob_y", "End_prob_likelihood",
    "Time_seconds", "Time_in_experiment", "Clock_time",
    "Tube", "RPi", "Video", "Round", "Behaviour"])
    data.to_csv(PATH_EXPORT, index=False)
    print(f"New file created at: {PATH_EXPORT}")

File found: /content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data_Oct-25.csv
Reading in dataframe
File successfully read.


Identify processed csvs (list the files from existing dataset)

>> Got to here

In [ ]:
def list_processed_videos(source_dir, search_string = "DLC_resnet50"):
    """
    Walks through the source directory and its subdirectories to find files containing the search_name in their names.

    :param source_dir: The root directory to start searching from.
    :param search_name: The substring to look for in file names. Default is "DLC_resnet50".
    :return: A list of file paths that contain the search_name in their names.
    """
    # List to hold the paths of matching files
    processed_videos = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        # Iterate over all files in the current root
        for file_name in files:
            if search_string in file_name:
                processed_file_path = os.path.join(root, file_name)
                # Extract the video name only from it
                video_name = processed_file_path.split('/Round 10/')[1].split('DLC')[0] + '.mp4'
                # Append the video name to the list if it is not a duplicate
                if video_name not in processed_videos:
                  processed_videos.append(video_name)

    print(len(processed_videos), "videos already processed and added to list")
    return processed_videos

processed_videos = list_processed_videos(directory_to_analyse_path)

IndexError: list index out of range

In [ ]:
def list_processed_csvs(all_csvs, folder_to_analyse):
    """
    Walks through the source directory and its subdirectories to find files containing the search_name in their names.

    :param source_dir: The root directory to start searching from.
    :param search_name: The substring to look for in file names. Default is "DLC_resnet50".
    :return: A list of file paths that contain the search_name in their names.
    """
    # List to hold the paths of matching files
    processed_csvs = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(folder_to_analyse):
        # Iterate over all files in the current root
        for file_name in files:
            if search_string in file_name:
                processed_file_path = os.path.join(root, file_name)
                # Extract the video name only from it
                video_name = processed_file_path.split('/Round 10/')[1].split('DLC')[0] + '.mp4'
                # Append the video name to the list if it is not a duplicate
                if video_name not in processed_videos:
                  processed_videos.append(video_name)

    print(len(processed_videos), "videos already processed and added to list")
    return processed_videos

processed_videos = list_processed_csvs(all_csvs, directory_to_analyse_path)

NameError: name 'search_string' is not defined

In [ ]:
already_processed = data['Video'].unique()

In [ ]:
len(already_processed)

37505

In [ ]:
# List all the unique video values in the file
already_processed = data['Video'].unique()
print("Already processed videos listed")
        else:
            print("The file does not contain a 'Video' column.")
    except Exception as e:
        print(f"Error reading file: {e}")
else:
    print("No file matching the criteria was found.")

In [ ]:
len(already_processed)

37505

In [ ]:
print(already_processed[1:10])

['left_06-11-2024_16-21-00DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-21-17DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-24-45DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-24-56DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-25-05DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-26-43DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_16-26-50DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'right_06-11-2024_16-55-07DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'
 'left_06-11-2024_17-16-57DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv']


List unprocessed csvs

In [ ]:
# Step 3: Identify unprocessed files
# Extract only filenames from all_csvs
all_csv_filenames = [os.path.basename(csv) for csv in all_csvs]

# Find files not already processed
unprocessed_csvs = [
    csv for csv in all_csvs
    if os.path.basename(csv) not in already_processed
]

print(f"{len(unprocessed_csvs)} CSV files to process")
#print(unprocessed_csvs[1:10])

2284 CSV files to process


In [ ]:
data['Video'].head() #get processed data and check is structure

,Video
0,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...
1,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...
2,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...
3,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...
4,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...


# Process unprocessed csvs

In [ ]:
# Directory to analyze
directory_list = os.listdir(directory_to_analyse_path)

# Initialize counter and data list
counter = 0

# Loop through directories
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    # Loop through folders
    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, directory, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]

        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(os.path.join(folder_to_analyse_path, 'Videos')) if file.endswith('.csv')]

        for file in csv_files:

            if file in already_processed:
                print("Skipping already processed file:", {file})
                continue

            elif file in unprocessed_csvs:
                file_path = os.path.join(folder_to_analyse_path, 'Videos', file)
                data_to_add = pandas.read_csv(file_path)
                data_to_add = data_to_add.drop([0, 1])  # Remove extraneous rows
                # Rename columns
                data_to_add.columns.values[0] = "Frame"
                data_to_add.columns.values[1] = "L_antenna_x"
                data_to_add.columns.values[2] = "L_antenna_y"
                data_to_add.columns.values[3] = "L_antenna_likelihood"
                data_to_add.columns.values[4] = "R_antenna_x"
                data_to_add.columns.values[5] = "R_antenna_y"
                data_to_add.columns.values[6] = "R_antenna_likelihood"
                data_to_add.columns.values[7] = "L_mandible_x"
                data_to_add.columns.values[8] = "L_mandible_y"
                data_to_add.columns.values[9] = "L_mandible_likelihood"
                data_to_add.columns.values[10] = "R_mandible_x"
                data_to_add.columns.values[11] = "R_mandible_y"
                data_to_add.columns.values[12] = "R_mandible_likelihood"
                data_to_add.columns.values[13] = "Top_prob_x"
                data_to_add.columns.values[14] = "Top_prob_y"
                data_to_add.columns.values[15] = "Top_prob_likelihood"
                data_to_add.columns.values[16] = "Tube_prob_x"
                data_to_add.columns.values[17] = "Tube_prob_y"
                data_to_add.columns.values[18] = "Tube_prob_likelihood"
                data_to_add.columns.values[19] = "End_prob_x"
                data_to_add.columns.values[20] = "End_prob_y"
                data_to_add.columns.values[21] = "End_prob_likelihood"
                # Add other metadata
                data_to_add["Tube"] = file[:4]
                data_to_add["RPi"] = pi_no
                data_to_add["Video"] = file
                data_to_add["Round"] = Round

                data = pd.concat([data, data_to_add], ignore_index=True)
                counter += 1
                print(f"{counter} CSV files processed")

# Concatenate and save the final dataset
print(f"{counter} csv files processed and added to dataframe")
data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24.csv', index=False)
print("Updated dataframe saved at " + '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24.csv')

Streaming output truncated to the last 5000 lines.
Skipping already processed file: {'right_15-11-2024_16-32-32DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-32-40DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-32-45DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-32-53DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-36-25DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-36-36DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-36-53DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-37-19DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_16-38-12DLC_resnet50_Beec

## Update the dataset to include all csvs

In [ ]:
def list_processed_videos(source_dir, search_string = "DLC_resnet50"):
    """
    Walks through the source directory and its subdirectories to find files containing the search_name in their names.

    :param source_dir: The root directory to start searching from.
    :param search_name: The substring to look for in file names. Default is "DLC_resnet50".
    :return: A list of file paths that contain the search_name in their names.
    """
    # List to hold the paths of matching files
    processed_videos = []

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        # Iterate over all files in the current root
        for file_name in files:
            if search_string in file_name:
                processed_file_path = os.path.join(root, file_name)
                # Extract the video name only from it
                video_name = processed_file_path.split('/Round 10/')[1].split('DLC')[0] + '.mp4'
                # Append the video name to the list if it is not a duplicate
                if video_name not in processed_videos:
                  processed_videos.append(video_name)

    print(len(processed_videos), "videos already processed and added to list")
    return processed_videos

processed_videos = list_processed_videos(folder_to_analyse_path)

In [ ]:
#List the number of csvs in each folder in the directory
import os

csv_counter = 0


directory_list = os.listdir(directory_to_analyse_path)

for directory in directory_list:
  if directory == 'Died':
    continue
  else:
    folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory)) #List of RPis in that round

  #FOR EACH FOLDER, extract info about when the pi started recording
  for folder in folder_list:
    #Make a list of csv files from videos that the pi recorded
    csv_files = [file for file in os.listdir(os.path.join(directory_to_analyse_path, directory, folder, 'Videos')) if file.endswith('.csv')]

    for file in csv_files:
      csv_counter += 1

print(csv_counter)

40179


In [ ]:
import os

# Define the directory containing extracted data
directory_to_analyse_path = '/content/drive/My Drive/VideoDrive/' + FOLDER_NAME

# Initialize a counter for CSV files
csv_counter = 0

# Walk through the directory tree
for root, dirs, files in os.walk(directory_to_analyse_path):
    # Check if we are currently inside a 'Videos' directory
    if 'Videos' in root:
        # List all files in the current 'Videos' directory
        print("Video folder located in", root, "and csvs listed")
        video_files = os.listdir(root)

        # Count the CSV files in the current Videos folder
        csv_counter += sum(1 for file in video_files if file.endswith('.csv'))

print(f"Total number of CSV files: {csv_counter}")

Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi10_06-11-2024_16-19-18/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi3_06-11-2024_15-58-16/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi6_06-11-2024_16-04-28/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi5_06-11-2024_16-01-19/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi1_06-11-2024_15-58-09/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi9_06-11-2024_16-11-14/Videos and csvs listed
Video folder located in /content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 1/Beecam_RPi7_06-11-

In [ ]:
import os
import pandas as pd

In [ ]:
import os
import pandas as pd

# Define the directory and file name pattern
directory = "/content/drive/My Drive/BeecamData/Other drugs/DLC_output"
file_substring = "all_data_Nov24"
target_file = None

# Check if any file in the directory contains the substring in its name
for file_name in os.listdir(directory):
    if file_substring in file_name:
        target_file = os.path.join(directory, file_name)
        break

# If the file exists, read it
if target_file:
    print(f"File found: {target_file}")
    try:
        data = pd.read_csv(target_file)
        print("File successfully read.")

        # List all the unique video values in the file
        if 'Video' in data.columns:
            already_processed = data['Video'].unique()
            print("Already processed videos listed")
        else:
            print("The file does not contain a 'Video' column.")
    except Exception as e:
        print(f"Error reading file: {e}")
else:
    print("No file matching the criteria was found.")


File found: /content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Nov24.csv
File successfully read.
Already processed videos listed


In [ ]:
already_processed

array(['left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv',
       'left_06-11-2024_16-21-00DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv',
       'left_06-11-2024_16-21-17DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv',
       ...,
       'left_17-11-2024_11-59-04DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv',
       'left_17-11-2024_13-17-33DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv',
       'left_17-11-2024_13-44-47DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'],
      dtype=object)

In [ ]:
import os
import pandas
from datetime import datetime

# Directory to analyze
directory_to_analyse_path = '/content/drive/My Drive/VideoDrive/' + FOLDER_NAME
directory_list = os.listdir(directory_to_analyse_path)

# Initialize counter and data list
counter = 0
all_data_list = pd.read_csv(target_file)

# Loop through directories
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    # Loop through folders
    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, directory, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]

        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(os.path.join(folder_to_analyse_path, 'Videos')) if file.endswith('.csv')]

        for file in csv_files:
            if file in already_processed:
                print("Skipping already processed file:", {file})
                continue

            file_path = os.path.join(folder_to_analyse_path, 'Videos', file)

            # Check for empty files
            if os.path.getsize(file_path) == 0:
                print(f"Skipping empty file: {file_path}")
                continue

            # Read the CSV file
            try:
                data = pandas.read_csv(file_path)
                data = data.drop([0, 1])  # Remove extraneous rows

                # Rename columns
                data.columns.values[0] = "Frame"
                data.columns.values[1] = "L_antenna_x"
                data.columns.values[2] = "L_antenna_y"
                data.columns.values[3] = "L_antenna_likelihood"
                data.columns.values[4] = "R_antenna_x"
                data.columns.values[5] = "R_antenna_y"
                data.columns.values[6] = "R_antenna_likelihood"
                data.columns.values[7] = "L_mandible_x"
                data.columns.values[8] = "L_mandible_y"
                data.columns.values[9] = "L_mandible_likelihood"
                data.columns.values[10] = "R_mandible_x"
                data.columns.values[11] = "R_mandible_y"
                data.columns.values[12] = "R_mandible_likelihood"
                data.columns.values[13] = "Top_prob_x"
                data.columns.values[14] = "Top_prob_y"
                data.columns.values[15] = "Top_prob_likelihood"
                data.columns.values[16] = "Tube_prob_x"
                data.columns.values[17] = "Tube_prob_y"
                data.columns.values[18] = "Tube_prob_likelihood"
                data.columns.values[19] = "End_prob_x"
                data.columns.values[20] = "End_prob_y"
                data.columns.values[21] = "End_prob_likelihood"

                # Add other metadata
                data["Tube"] = file[:4]
                data["RPi"] = pi_no
                data["Video"] = file
                data["Round"] = Round

                all_data_list.append(data)
                counter += 1
                print(f"{counter} CSV files processed")

            except Exception as e:
                print(f"Error processing file {file}: {e}")

# Concatenate and save the final dataset
all_data = pandas.concat(all_data_list, ignore_index=True)
all_data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Nov24.csv', index=False)

print("Data extraction complete.")


Streaming output truncated to the last 5000 lines.
Skipping already processed file: {'right_15-11-2024_18-48-09DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-48-27DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-48-41DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-48-54DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-49-02DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-53-03DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-53-24DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'right_15-11-2024_18-53-33DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv'}
Skipping already processed file: {'left_15-11-2024_18-53-59DLC_resnet50_Beeca

KeyboardInterrupt: 

In [ ]:
import os
import pandas
from datetime import datetime, timedelta
import fnmatch

counter = 0

#ACTION REQUIRED: DEFINE DIRECTORY CONTAINING EXTRACTED DATA
directory_to_analyse_path = '/content/drive/My Drive/VideoDrive/' + FOLDER_NAME
directory_list = os.listdir(directory_to_analyse_path)

#RENAME: Create a master sheet with absolute time in the left column, an RPi variable, a tube variable, and a behaviour column
#Quinine_data = pandas.DataFrame(columns=['Round', 'RPi','Tube', 'Frame', 'Time_in_experiment'])
all_data_list = []
#Populate this by looping through the folders

for directory in directory_list:
  #unless directory is called "Died", set Round equal to the 7th character in the directory name
  if directory == 'Died':
    continue
  else:
    Round = directory[6:]
    folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory)) #List of RPis in that round

  #FOR EACH FOLDER, extract info about when the pi started recording
  for folder in folder_list:
    folder_to_analyse_path = os.path.join(directory_to_analyse_path, folder)
    pi_no = folder[7:12] #Start bit of folder name
    if pi_no == 'RPi10':
      start_date = folder[13:23]
      start_time = folder[24:32]
    else:
      start_date = folder[12:22]
      start_time = folder[23:31] #End bit of folder name
    #Extract date and time information
    YEAR_g = start_date[6:10]
    MONTH_g = start_date[3:5]
    DAY_g = start_date[0:2]
    HOUR_g = start_time[0:2]
    MIN_g = start_time[3:5]
    SEC_g = start_time[6:8]
    start_time_g = start_time #= folder[23:31] (i.e. the global start time - when the RPi was switched on)
    start_time_g_object = datetime.strptime(start_time_g, '%H-%M-%S').time()

    #Make a list of csv files from videos that the pi recorded
    csv_files = [file for file in os.listdir(os.path.join(directory_to_analyse_path, directory, folder, 'Videos')) if file.endswith('.csv')]

    for file in csv_files:
      file_path = os.path.join(directory_to_analyse_path, directory, folder, 'Videos', file)
      # Before reading the CSV file
      if os.path.getsize(file_path) > 0:
          data = pandas.read_csv(file_path)
          data = data.drop([0, 1])  # Remove the extraneous two rows above
          # Rename each of the columns as before
      else:
          print(f"Skipping empty file: {file_path}")
          continue  # Move to the next file in the loop
    #Extract tube info and date/time from each of them
      tube = file[0:5]
      if tube == 'left_':
        tube = 'left'
        start_date_bout = file[5:15]
        start_time_bout = file[16:24] #Left vs right means not in the same place in the string
      elif tube == 'right':
        start_date_bout = file[6:16]
        start_time_bout = file[17:25]
      YEAR_b = start_date_bout[6:10]
      MONTH_b = start_date_bout[3:5]
      DAY_b = start_date_bout[0:2]
      HOUR_b = start_time_bout[0:2]
      MIN_b = start_time_bout[3:5]
      SEC_b = start_time_bout[6:8]
      start_time_b = start_time_bout ## = video[17:25] (when the individual video recording began)
      start_time_b_datetime = datetime(int(YEAR_b), int(MONTH_b), int(DAY_b), int(HOUR_b), int(MIN_b), int(SEC_b))
      start_time_b_object = datetime.strptime(start_time_b, '%H-%M-%S').time()

      #Read each csv file and open it with pandas to create a dataframe called 'data'
      data = pandas.read_csv(file_path)
      data = data.drop([0, 1]) #Remove the extraneous two rows above
      #Rename each of the columns
      data.columns.values[0] = "Frame"
      data.columns.values[1] = "L_antenna_x"
      data.columns.values[2] = "L_antenna_y"
      data.columns.values[3] = "L_antenna_likelihood"
      data.columns.values[4] = "R_antenna_x"
      data.columns.values[5] = "R_antenna_y"
      data.columns.values[6] = "R_antenna_likelihood"
      data.columns.values[7] = "L_mandible_x"
      data.columns.values[8] = "L_mandible_y"
      data.columns.values[9] = "L_mandible_likelihood"
      data.columns.values[10] = "R_mandible_x"
      data.columns.values[11] = "R_mandible_y"
      data.columns.values[12] = "R_mandible_likelihood"
      data.columns.values[13] = "Top_prob_x"
      data.columns.values[14] = "Top_prob_y"
      data.columns.values[15] = "Top_prob_likelihood"
      data.columns.values[16] = "Tube_prob_x"
      data.columns.values[17] = "Tube_prob_y"
      data.columns.values[18] = "Tube_prob_likelihood"
      data.columns.values[19] = "End_prob_x"
      data.columns.values[20] = "End_prob_y"
      data.columns.values[21] = "End_prob_likelihood"
      #TIME VARIABLES (NB: global time requires that the pi was offline the whole way through the experiment)
      # Convert Frame column to numeric, with errors set to NaN for any non-numeric values
      data = data.dropna(subset=["Frame"]) #Remove any rows where Frame is NaN
      data["Frame"] = pandas.to_numeric(data["Frame"], errors="coerce")
      # Replace any NaNs in Frame (from non-numeric values) with 0 or remove those rows
      # Now perform the calculation
      data["Time_seconds"] = data["Frame"] * 0.2 #Convert frame number to absolute time by adding 0.2*frame seconds to start time of the video
      difference = datetime.strptime(str(start_time_b_object), '%H:%M:%S') - datetime.strptime(str(start_time_g_object), '%H:%M:%S')
      data.at[0, "Time_in_experiment"] = difference
      # Update 'Time_in_experiment' calculation using pd.to_timedelta
      data['Time_in_experiment'] = difference + pandas.to_timedelta(data['Time_seconds'], unit='s')
      # Update 'Clock_time' calculation using pd.to_timedelta
      data["Clock_time"] = start_time_b_datetime
      data['Clock_time'] = start_time_b_datetime + pandas.to_timedelta(data['Time_seconds'], unit='s')
      #Add other reference variables
      data["Tube"] = tube
      data["RPi"] = pi_no
      data["Video"] = file
      data["Round"] = Round

      all_data_list.append(data)
      counter += 1
      print(counter, "csv files processed")

print("Data extraction complete")
# Concatenate all data at once
all_data = pandas.concat(all_data_list, ignore_index=True)
all_data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Nov24.csv', index=False)



# Select only the required columns
required_columns = [0, 17, 20, 22, 23, 24, 25, 26, 27, 28, 18, 21]
proboscis_data = all_data.iloc[:, required_columns].copy()

# Convert columns to numeric where necessary, coercing errors to NaN
proboscis_data["Tube_prob_likelihood"] = pandas.to_numeric(proboscis_data["Tube_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_likelihood"] = pandas.to_numeric(proboscis_data["End_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_y"] = pandas.to_numeric(proboscis_data["End_prob_y"], errors='coerce')
proboscis_data["Tube_prob_y"] = pandas.to_numeric(proboscis_data["Tube_prob_y"], errors='coerce')

# Filter rows where:
#   1. Tube_prob_likelihood or End_prob_likelihood is greater than 0.6
#   2. End_prob_y is >= 10
#   3. Tube_prob_y is >= 10
proboscis_data = proboscis_data[
    ((proboscis_data["Tube_prob_likelihood"] > 0.6) | (proboscis_data["End_prob_likelihood"] > 0.6)) &
    (proboscis_data["End_prob_y"] >= 10) &
    (proboscis_data["Tube_prob_y"] >= 10)
].copy()

# Define the function to classify behavior
def get_behaviour(row):
    behaviour = 'No_prob'
    if (row['Tube_prob_likelihood'] >= 0.6) & (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Drinking"
    elif (row['Tube_prob_likelihood'] >= 0.6) ^ (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Prob_out"
    return behaviour

proboscis_data['Behaviour'] = proboscis_data.apply(get_behaviour, axis=1)
proboscis_data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data.csv', index=False)

1 csv files processed
2 csv files processed
3 csv files processed
4 csv files processed
5 csv files processed
6 csv files processed
7 csv files processed
8 csv files processed
9 csv files processed
10 csv files processed
11 csv files processed
12 csv files processed
13 csv files processed
14 csv files processed
15 csv files processed
16 csv files processed
17 csv files processed
18 csv files processed
19 csv files processed
20 csv files processed
21 csv files processed
22 csv files processed
23 csv files processed
24 csv files processed
25 csv files processed
26 csv files processed
27 csv files processed
28 csv files processed
29 csv files processed
30 csv files processed
31 csv files processed
32 csv files processed
33 csv files processed
34 csv files processed
35 csv files processed
36 csv files processed
37 csv files processed
38 csv files processed
39 csv files processed
40 csv files processed
41 csv files processed
42 csv files processed
43 csv files processed
44 csv files process

<ipython-input-2-6b68380d01a7>:52: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)
<ipython-input-2-6b68380d01a7>:78: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)


912 csv files processed
913 csv files processed
914 csv files processed
915 csv files processed
916 csv files processed
917 csv files processed
918 csv files processed
919 csv files processed
920 csv files processed
921 csv files processed
922 csv files processed
923 csv files processed
924 csv files processed
925 csv files processed
926 csv files processed
927 csv files processed
928 csv files processed
929 csv files processed
930 csv files processed


<ipython-input-2-6b68380d01a7>:52: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)
<ipython-input-2-6b68380d01a7>:78: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)


931 csv files processed
932 csv files processed
933 csv files processed
934 csv files processed
935 csv files processed
936 csv files processed
937 csv files processed
938 csv files processed
939 csv files processed
940 csv files processed
941 csv files processed
942 csv files processed
943 csv files processed
944 csv files processed
945 csv files processed
946 csv files processed
947 csv files processed
948 csv files processed
949 csv files processed
950 csv files processed
951 csv files processed
952 csv files processed
953 csv files processed
954 csv files processed
955 csv files processed
956 csv files processed
957 csv files processed
958 csv files processed
959 csv files processed
960 csv files processed
961 csv files processed
962 csv files processed
963 csv files processed
964 csv files processed
965 csv files processed
966 csv files processed
967 csv files processed
968 csv files processed
969 csv files processed
970 csv files processed
971 csv files processed
972 csv files pr

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


1349 csv files processed
1350 csv files processed
1351 csv files processed
1352 csv files processed
1353 csv files processed
1354 csv files processed
1355 csv files processed
1356 csv files processed
1357 csv files processed
1358 csv files processed
1359 csv files processed
1360 csv files processed
1361 csv files processed
1362 csv files processed
1363 csv files processed
1364 csv files processed
1365 csv files processed
1366 csv files processed
1367 csv files processed
1368 csv files processed
1369 csv files processed
1370 csv files processed
1371 csv files processed
1372 csv files processed
1373 csv files processed
1374 csv files processed
1375 csv files processed
1376 csv files processed
1377 csv files processed
1378 csv files processed
1379 csv files processed
1380 csv files processed
1381 csv files processed
1382 csv files processed
1383 csv files processed
1384 csv files processed
1385 csv files processed
1386 csv files processed
1387 csv files processed
1388 csv files processed


<ipython-input-2-6b68380d01a7>:52: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)
<ipython-input-2-6b68380d01a7>:78: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pandas.read_csv(file_path)


2131 csv files processed
2132 csv files processed
2133 csv files processed
2134 csv files processed
2135 csv files processed
2136 csv files processed
2137 csv files processed
2138 csv files processed
2139 csv files processed
2140 csv files processed
2141 csv files processed
2142 csv files processed
2143 csv files processed
2144 csv files processed
2145 csv files processed
2146 csv files processed
2147 csv files processed
2148 csv files processed
2149 csv files processed
2150 csv files processed
2151 csv files processed
2152 csv files processed
2153 csv files processed
2154 csv files processed
2155 csv files processed
2156 csv files processed
2157 csv files processed
2158 csv files processed
2159 csv files processed
2160 csv files processed
2161 csv files processed
2162 csv files processed
2163 csv files processed
2164 csv files processed
2165 csv files processed
2166 csv files processed
2167 csv files processed
2168 csv files processed
2169 csv files processed
2170 csv files processed


KeyboardInterrupt: 

In [ ]:
all_data = pd.read_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24.csv')

# Select only the required columns
required_columns = [0, 17, 20, 22, 23, 24, 25, 26, 27, 28, 18, 21]
proboscis_data = all_data.iloc[:, required_columns].copy()

# Convert columns to numeric where necessary, coercing errors to NaN
proboscis_data["Tube_prob_likelihood"] = pandas.to_numeric(proboscis_data["Tube_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_likelihood"] = pandas.to_numeric(proboscis_data["End_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_y"] = pandas.to_numeric(proboscis_data["End_prob_y"], errors='coerce')
proboscis_data["Tube_prob_y"] = pandas.to_numeric(proboscis_data["Tube_prob_y"], errors='coerce')

# Filter rows where:
#   1. Tube_prob_likelihood or End_prob_likelihood is greater than 0.6
#   2. End_prob_y is >= 10
#   3. Tube_prob_y is >= 10
proboscis_data = proboscis_data[
    ((proboscis_data["Tube_prob_likelihood"] > 0.6) | (proboscis_data["End_prob_likelihood"] > 0.6)) &
    (proboscis_data["End_prob_y"] >= 10) &
    (proboscis_data["Tube_prob_y"] >= 10)
].copy()

# Define the function to classify behavior
def get_behaviour(row):
    behaviour = 'No_prob'
    if (row['Tube_prob_likelihood'] >= 0.6) & (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Drinking"
    elif (row['Tube_prob_likelihood'] >= 0.6) ^ (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Prob_out"
    return behaviour

proboscis_data['Behaviour'] = proboscis_data.apply(get_behaviour, axis=1)
proboscis_data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data.csv', index=False)

<ipython-input-4-32a192230dc7>:1: DtypeWarning: Columns (22,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24.csv')


In [ ]:
import pandas as pd

# File paths
input_file = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24.csv'
output_file = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data.csv'

# Columns to load
required_columns = [0, 17, 20, 22, 23, 24, 25, 26, 27, 28, 18, 21]
columns_to_numeric = ["Tube_prob_likelihood", "End_prob_likelihood", "End_prob_y", "Tube_prob_y"]

# Function to classify behavior
def get_behaviour(row):
    behaviour = 'No_prob'
    if (row['Tube_prob_likelihood'] >= 0.6) & (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Drinking"
    elif (row['Tube_prob_likelihood'] >= 0.6) ^ (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Prob_out"
    return behaviour

# Process the file in chunks
chunk_size = 100000  # Adjust based on your available memory
chunks = pd.read_csv(input_file, usecols=required_columns, chunksize=chunk_size)

# Initialize output file
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}...")

    # Convert columns to numeric where necessary
    for col in columns_to_numeric:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce')

    # Apply filtering criteria
    chunk = chunk[
        ((chunk["Tube_prob_likelihood"] > 0.6) | (chunk["End_prob_likelihood"] > 0.6)) &
        (chunk["End_prob_y"] >= 10) &
        (chunk["Tube_prob_y"] >= 10)
    ].copy()

    # Classify behavior
    chunk['Behaviour'] = chunk.apply(get_behaviour, axis=1)

    # Append to output file
    chunk.to_csv(output_file, mode='a', index=False)

print(f"Processing complete. Filtered data saved to {output_file}.")


Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 38...
Processing chunk 39...
Processing chunk 40...
Processing chunk 41...
Processing chunk 42...
Processing chunk 43...
Processing chunk 44.

<ipython-input-1-e4fbf4fdab43>:25: DtypeWarning: Columns (22,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunks):


Processing chunk 90...
Processing chunk 91...
Processing chunk 92...
Processing chunk 93...
Processing chunk 94...
Processing chunk 95...
Processing chunk 96...
Processing chunk 97...
Processing chunk 98...
Processing chunk 99...
Processing chunk 100...
Processing chunk 101...
Processing chunk 102...
Processing chunk 103...
Processing chunk 104...
Processing chunk 105...
Processing chunk 106...
Processing chunk 107...
Processing chunk 108...
Processing chunk 109...
Processing chunk 110...
Processing chunk 111...
Processing chunk 112...
Processing chunk 113...
Processing chunk 114...
Processing chunk 115...
Processing chunk 116...
Processing chunk 117...
Processing chunk 118...
Processing chunk 119...
Processing chunk 120...
Processing chunk 121...
Processing chunk 122...
Processing chunk 123...
Processing chunk 124...
Processing chunk 125...
Processing chunk 126...
Processing chunk 127...
Processing chunk 128...
Processing chunk 129...
Processing chunk 130...
Processing chunk 131...
Pr

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Define directory containing extracted data
directory_to_analyse_path = '/content/drive/My Drive/VideoDrive/' + FOLDER_NAME
directory_list = os.listdir(directory_to_analyse_path)

# Output file paths
output_file = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Nov24.csv'
proboscis_file = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data.csv'

# Write headers to the output file before starting the loop
columns = [
    "Frame", "L_antenna_x", "L_antenna_y", "L_antenna_likelihood",
    "R_antenna_x", "R_antenna_y", "R_antenna_likelihood",
    "L_mandible_x", "L_mandible_y", "L_mandible_likelihood",
    "R_mandible_x", "R_mandible_y", "R_mandible_likelihood",
    "Top_prob_x", "Top_prob_y", "Top_prob_likelihood",
    "Tube_prob_x", "Tube_prob_y", "Tube_prob_likelihood",
    "End_prob_x", "End_prob_y", "End_prob_likelihood",
    "Time_seconds", "Time_in_experiment", "Clock_time",
    "Tube", "RPi", "Video", "Round"
]
pd.DataFrame(columns=columns).to_csv(output_file, index=False)

counter = 0

# Loop through directories and process files
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]

        start_time_g_object = datetime.strptime(start_time, '%H-%M-%S').time()
        csv_files = [file for file in os.listdir(os.path.join(directory_to_analyse_path, directory, folder, 'Videos')) if file.endswith('.csv')]

        for file in csv_files:
            file_path = os.path.join(directory_to_analyse_path, directory, folder, 'Videos', file)
            if os.path.getsize(file_path) > 0:
                data = pd.read_csv(file_path)
                data = data.drop([0, 1])
            else:
                print(f"Skipping empty file: {file_path}")
                continue

            tube = file[0:5]
            if tube == 'left_':
                tube = 'left'
                start_date_bout = file[5:15]
                start_time_bout = file[16:24]
            elif tube == 'right':
                start_date_bout = file[6:16]
                start_time_bout = file[17:25]
            else:
                continue

            start_time_b_datetime = datetime.strptime(f"{start_date_bout} {start_time_bout}", '%d-%m-%Y %H-%M-%S')

            data.columns = columns[:22]  # Rename columns
            data["Time_seconds"] = data["Frame"].astype(float) * 0.2
            difference = timedelta(hours=start_time_g_object.hour, minutes=start_time_g_object.minute, seconds=start_time_g_object.second) - timedelta(hours=start_time_b_datetime.hour, minutes=start_time_b_datetime.minute, seconds=start_time_b_datetime.second)
            data["Time_in_experiment"] = difference + pd.to_timedelta(data["Time_seconds"], unit='s')
            data["Clock_time"] = start_time_b_datetime + pd.to_timedelta(data["Time_seconds"], unit='s')
            data["Tube"] = tube
            data["RPi"] = pi_no
            data["Video"] = file
            data["Round"] = Round

            # Append to CSV
            data.to_csv(output_file, mode='a', index=False, header=False)

            counter += 1
            print(f"{counter} csv files processed")

print("Data extraction complete.")


1 csv files processed
2 csv files processed
3 csv files processed
4 csv files processed
5 csv files processed
6 csv files processed
7 csv files processed
8 csv files processed
9 csv files processed
10 csv files processed
11 csv files processed
12 csv files processed
13 csv files processed
14 csv files processed
15 csv files processed
16 csv files processed
17 csv files processed
18 csv files processed
19 csv files processed
20 csv files processed
21 csv files processed
22 csv files processed
23 csv files processed
24 csv files processed
25 csv files processed
26 csv files processed
27 csv files processed
28 csv files processed
29 csv files processed
30 csv files processed
31 csv files processed
32 csv files processed
33 csv files processed
34 csv files processed
35 csv files processed
36 csv files processed
37 csv files processed
38 csv files processed
39 csv files processed
40 csv files processed
41 csv files processed
42 csv files processed
43 csv files processed
44 csv files process

<ipython-input-3-0d0ad24a1a74>:53: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


912 csv files processed
913 csv files processed
914 csv files processed
915 csv files processed
916 csv files processed
917 csv files processed
918 csv files processed
919 csv files processed
920 csv files processed
921 csv files processed
922 csv files processed
923 csv files processed
924 csv files processed
925 csv files processed
926 csv files processed
927 csv files processed
928 csv files processed
929 csv files processed
930 csv files processed


<ipython-input-3-0d0ad24a1a74>:53: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


931 csv files processed
932 csv files processed
933 csv files processed
934 csv files processed
935 csv files processed
936 csv files processed
937 csv files processed
938 csv files processed
939 csv files processed
940 csv files processed
941 csv files processed
942 csv files processed
943 csv files processed
944 csv files processed
945 csv files processed
946 csv files processed
947 csv files processed
948 csv files processed
949 csv files processed
950 csv files processed
951 csv files processed
952 csv files processed
953 csv files processed
954 csv files processed
955 csv files processed
956 csv files processed
957 csv files processed
958 csv files processed
959 csv files processed
960 csv files processed
961 csv files processed
962 csv files processed
963 csv files processed
964 csv files processed
965 csv files processed
966 csv files processed
967 csv files processed
968 csv files processed
969 csv files processed
970 csv files processed
971 csv files processed
972 csv files pr

<ipython-input-3-0d0ad24a1a74>:53: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Streaming output truncated to the last 5000 lines.
32877 csv files processed
32878 csv files processed
32879 csv files processed
32880 csv files processed
32881 csv files processed
32882 csv files processed
32883 csv files processed
32884 csv files processed
32885 csv files processed
32886 csv files processed
32887 csv files processed
32888 csv files processed
32889 csv files processed
32890 csv files processed
32891 csv files processed
32892 csv files processed
32893 csv files processed
32894 csv files processed
32895 csv files processed
32896 csv files processed
32897 csv files processed
32898 csv files processed
32899 csv files processed
32900 csv files processed
32901 csv files processed
32902 csv files processed
32903 csv files processed
32904 csv files processed
32905 csv files processed
32906 csv files processed
32907 csv files processed
32908 csv files processed
32909 csv files processed
32910 csv files processed
32911 csv files processed
32912 csv files processed
32913 csv fil

OSError: [Errno 5] Input/output error: '/content/drive/My Drive/VideoDrive/Other drugs November 2024/Round 11/Beecam_RPi1_16-11-2024_15-37-49/Videos'

In [ ]:
#Find number of rows
all_data_list = pd.read_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Nov24.csv')
print(len(all_data_list))

8984685


In [ ]:
all_data_list[0].head()

,Frame,L_antenna_x,L_antenna_y,L_antenna_likelihood,R_antenna_x,R_antenna_y,R_antenna_likelihood,L_mandible_x,L_mandible_y,L_mandible_likelihood,...,End_prob_x,End_prob_y,End_prob_likelihood,Time_seconds,Time_in_experiment,Clock_time,Tube,RPi,Video,Round
2,0.0,3.6187169551849365,30.161367416381836,0.0005597984418272972,239.29373168945312,4.4332380294799805,0.0016397341387346387,239.35205078125,1.512887954711914,6.677553028566763e-05,...,241.11509704589844,1.0415675640106201,0.0004756973939947784,0.0,0 days 00:01:14,2024-11-06 16:20:32.000,left,RPi10,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...,1
3,1.0,3.7133283615112305,32.786720275878906,0.0009531785035505891,239.15643310546875,4.512442111968994,0.0016791742527857423,222.3592987060547,1.8967328071594238,7.088975689839572e-05,...,240.9250946044922,1.0897266864776611,0.0004659210972022265,0.2,0 days 00:01:14.200000,2024-11-06 16:20:32.200,left,RPi10,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...,1
4,2.0,3.88200306892395,31.2332706451416,0.0008263784111477435,239.0331268310547,4.386338233947754,0.0013752005761489272,222.43763732910156,1.92911958694458,7.097722846083343e-05,...,240.8111114501953,0.9542648792266846,0.00043955101864412427,0.4,0 days 00:01:14.400000,2024-11-06 16:20:32.400,left,RPi10,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...,1
5,3.0,3.4369163513183594,32.444828033447266,0.0008839276270009577,239.20291137695312,4.4793477058410645,0.0013516813050955534,222.3954315185547,1.915323257446289,7.133570761652663e-05,...,240.99313354492188,1.0131278038024902,0.00038709898944944143,0.6,0 days 00:01:14.600000,2024-11-06 16:20:32.600,left,RPi10,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...,1
6,4.0,3.3534228801727295,32.44471740722656,0.0007787076174281538,239.280029296875,4.344605445861816,0.00133195158559829,222.5293426513672,1.9017295837402344,6.732720794389024e-05,...,241.1986846923828,0.8695511817932129,0.00038867254625074565,0.8,0 days 00:01:14.800000,2024-11-06 16:20:32.800,left,RPi10,left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4...,1


In [ ]:
all_data = pandas.concat(all_data_list, ignore_index=True)

In [ ]:
all_data.to_csv('/content/drive/My Drive/BeecamData/Quinine pilot/DLC_output/all_data_Nov24.csv', index=False)

Just proboscis data from all frames

In [ ]:
#Produce subset of all_data called proboscis_data which contains Tube_prob_likelihoods > 0.6 and/or End_prob_likelihoods > 0.6
#required_columns = [0, 22, 23, 24, 25, 26, 27, 28, 18, 21]
#proboscis_data = all_data.iloc[:, required_columns].copy()
#proboscis_data = all_data[(proboscis_data["Tube_prob_likelihood"] > 0.6) | (proboscis_data["End_prob_likelihood"] > 0.6)].copy()

In [ ]:
# Select only the required columns
required_columns = [0, 17, 20, 22, 23, 24, 25, 26, 27, 28, 18, 21]
proboscis_data = all_data.iloc[:, required_columns].copy()

# Convert columns to numeric where necessary, coercing errors to NaN
proboscis_data["Tube_prob_likelihood"] = pandas.to_numeric(proboscis_data["Tube_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_likelihood"] = pandas.to_numeric(proboscis_data["End_prob_likelihood"], errors='coerce')
proboscis_data["End_prob_y"] = pandas.to_numeric(proboscis_data["End_prob_y"], errors='coerce')
proboscis_data["Tube_prob_y"] = pandas.to_numeric(proboscis_data["Tube_prob_y"], errors='coerce')

# Filter rows where:
#   1. Tube_prob_likelihood or End_prob_likelihood is greater than 0.6
#   2. End_prob_y is >= 10
#   3. Tube_prob_y is >= 10
proboscis_data = proboscis_data[
    ((proboscis_data["Tube_prob_likelihood"] > 0.6) | (proboscis_data["End_prob_likelihood"] > 0.6)) &
    (proboscis_data["End_prob_y"] >= 10) &
    (proboscis_data["Tube_prob_y"] >= 10)
].copy()


Define behaviour classification function

In [ ]:
# Define the function to classify behavior
def get_behaviour(row):
    behaviour = 'No_prob'
    if (row['Tube_prob_likelihood'] >= 0.6) & (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Drinking"
    elif (row['Tube_prob_likelihood'] >= 0.6) ^ (row['End_prob_likelihood'] >= 0.6):
        behaviour = "Prob_out"
    return behaviour

Just frames containing end of probosics ± intersection with tube

In [ ]:
proboscis_data['Behaviour'] = proboscis_data.apply(get_behaviour, axis=1)
proboscis_data.to_csv('/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data.csv', index=False)

Just drinking data

In [ ]:
drinking_data = proboscis_data[(proboscis_data["Tube_prob_likelihood"] > 0.6) & (proboscis_data["End_prob_likelihood"] > 0.6)]
drinking_data.to_csv('/content/drive/My Drive/BeecamData/' + FOLDER_NAME + '/drinking_data.csv', index=False)

---




# List processed csvs

# Get proboscis data

### Create proboscis_data output file, with headers

In [ ]:
import csv

# Define the column headers
columns = [
    "Frame", "L_antenna_x", "L_antenna_y", "L_antenna_likelihood",
    "R_antenna_x", "R_antenna_y", "R_antenna_likelihood",
    "L_mandible_x", "L_mandible_y", "L_mandible_likelihood",
    "R_mandible_x", "R_mandible_y", "R_mandible_likelihood",
    "Top_prob_x", "Top_prob_y", "Top_prob_likelihood",
    "Tube_prob_x", "Tube_prob_y", "Tube_prob_likelihood",
    "End_prob_x", "End_prob_y", "End_prob_likelihood",
    "Time_seconds", "Time_in_experiment", "Clock_time",
    "Tube", "RPi", "Video", "Round", "Behaviour"
]

# Create a new CSV file and write the column headers
with open(PATH_EXPORT, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(columns)

print(f"CSV file '{PATH_EXPORT}' created with column headers.")

CSV file '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/proboscis_data_Oct-25.csv' created with column headers.


### Read csvs, extracting relevant frames and writing to this file

In [ ]:
file_exists = os.path.exists(PATH_EXPORT)

# Directory to analyze
directory_list = os.listdir(directory_to_analyse_path)
# Initialize counter
counter = 0

# Define the function to classify behavior
def get_behaviour(row):
    if (row['Tube_prob_likelihood'] >= 0.8) & (row['End_prob_likelihood'] >= 0.8):
        behaviour = "Drinking"
    elif (row['Tube_prob_likelihood'] >= 0.8) & (row['End_prob_likelihood'] < 0.8):
      behaviour = "Drinking_no_end"
    elif (row['Tube_prob_likelihood'] < 0.8) & (row['End_prob_likelihood'] >= 0.8):
        behaviour = "Prob_out"
    else:
      behaviour = "No_prob"
    return behaviour

# Loop through directories
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    # Loop through folders
    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, directory, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]
        #Extract date and time information
        YEAR_g = start_date[6:10]
        MONTH_g = start_date[3:5]
        DAY_g = start_date[0:2]
        HOUR_g = start_time[0:2]
        MIN_g = start_time[3:5]
        SEC_g = start_time[6:8]
        start_time_g = start_time #= folder[23:31] (i.e. the global start time - when the RPi was switched on)
        start_time_g_object = datetime.strptime(start_time_g, '%H-%M-%S').time()

        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(os.path.join(folder_to_analyse_path, 'Videos')) if file.endswith('.csv')]
        #OPEN THE FILE
        for file in csv_files:
            counter += 1
            print(f"Processing CSV {counter} in list: {file}")
            file_path = os.path.join(folder_to_analyse_path, 'Videos', file)

            #READ IN THE CSV AND SELECT BITS OF INTEREST
            new_data = pd.read_csv(file_path)
            proboscis_data = new_data.copy() #proboscis_data = new_data.iloc[:, required_columns].copy()
            proboscis_data_to_add = proboscis_data.drop([0, 1])  # Remove extraneous rows (i.e. the two rows that contain headers, so just numbers left)
            tube = file[0:5]
            if tube == 'left_':
              tube = 'left'
              start_date_bout = file[5:15]
              start_time_bout = file[16:24] #Left vs right means not in the same place in the string
            elif tube == 'right':
              start_date_bout = file[6:16]
              start_time_bout = file[17:25]
            YEAR_b = start_date_bout[6:10]
            MONTH_b = start_date_bout[3:5]
            DAY_b = start_date_bout[0:2]
            HOUR_b = start_time_bout[0:2]
            MIN_b = start_time_bout[3:5]
            SEC_b = start_time_bout[6:8]
            start_time_b = start_time_bout ## = video[17:25] (when the individual video recording began)
            start_time_b_datetime = datetime(int(YEAR_b), int(MONTH_b), int(DAY_b), int(HOUR_b), int(MIN_b), int(SEC_b))
            start_time_b_object = datetime.strptime(start_time_b, '%H-%M-%S').time()
            # Rename columns
            proboscis_data_to_add.columns.values[0] = "Frame"
            proboscis_data_to_add.columns.values[1] = "L_antenna_x"
            proboscis_data_to_add.columns.values[2] = "L_antenna_y"
            proboscis_data_to_add.columns.values[3] = "L_antenna_likelihood"
            proboscis_data_to_add.columns.values[4] = "R_antenna_x"
            proboscis_data_to_add.columns.values[5] = "R_antenna_y"
            proboscis_data_to_add.columns.values[6] = "R_antenna_likelihood"
            proboscis_data_to_add.columns.values[7] = "L_mandible_x"
            proboscis_data_to_add.columns.values[8] = "L_mandible_y"
            proboscis_data_to_add.columns.values[9] = "L_mandible_likelihood"
            proboscis_data_to_add.columns.values[10] = "R_mandible_x"
            proboscis_data_to_add.columns.values[11] = "R_mandible_y"
            proboscis_data_to_add.columns.values[12] = "R_mandible_likelihood"
            proboscis_data_to_add.columns.values[13] = "Top_prob_x"
            proboscis_data_to_add.columns.values[14] = "Top_prob_y"
            proboscis_data_to_add.columns.values[15] = "Top_prob_likelihood"
            proboscis_data_to_add.columns.values[16] = "Tube_prob_x"
            proboscis_data_to_add.columns.values[17] = "Tube_prob_y"
            proboscis_data_to_add.columns.values[18] = "Tube_prob_likelihood"
            proboscis_data_to_add.columns.values[19] = "End_prob_x"
            proboscis_data_to_add.columns.values[20] = "End_prob_y"
            proboscis_data_to_add.columns.values[21] = "End_prob_likelihood"
            #Add time variables and sort out frame variable
            # Convert Frame column to numeric, with errors set to NaN for any non-numeric values
            proboscis_data_to_add = proboscis_data_to_add.dropna(subset=["Frame"]) #Remove any rows where Frame is NaN
            proboscis_data_to_add["Frame"] = pd.to_numeric(proboscis_data_to_add["Frame"], errors="coerce")
            # Replace any NaNs in Frame (from non-numeric values) with 0 or remove those rows
            # Now perform the calculation
            proboscis_data_to_add["Time_seconds"] = proboscis_data_to_add["Frame"] * 0.2 #Convert frame number to absolute time by adding 0.2*frame seconds to start time of the video
            difference = datetime.strptime(str(start_time_b_object), '%H:%M:%S') - datetime.strptime(str(start_time_g_object), '%H:%M:%S')
            proboscis_data_to_add.at[0, "Time_in_experiment"] = difference
            # Update 'Time_in_experiment' calculation using pd.to_timedelta
            proboscis_data_to_add['Time_in_experiment'] = difference + pd.to_timedelta(proboscis_data_to_add['Time_seconds'], unit='s')
            # Update 'Clock_time' calculation using pd.to_timedelta
            proboscis_data_to_add["Clock_time"] = start_time_b_datetime
            proboscis_data_to_add['Clock_time'] = start_time_b_datetime + pd.to_timedelta(proboscis_data_to_add['Time_seconds'], unit='s')
            # Add metadata columns
            proboscis_data_to_add["Tube"] = tube
            proboscis_data_to_add["RPi"] = pi_no
            proboscis_data_to_add["Video"] = file
            proboscis_data_to_add["Round"] = Round
            # Convert columns to numeric where necessary, coercing errors to NaN
            proboscis_data_to_add["Tube_prob_likelihood"] = pd.to_numeric(proboscis_data_to_add["Tube_prob_likelihood"], errors='coerce')
            proboscis_data_to_add["End_prob_likelihood"] = pd.to_numeric(proboscis_data_to_add["End_prob_likelihood"], errors='coerce')
            proboscis_data_to_add["End_prob_y"] = pd.to_numeric(proboscis_data_to_add["End_prob_y"], errors='coerce')
            proboscis_data_to_add["Tube_prob_y"] = pd.to_numeric(proboscis_data_to_add["Tube_prob_y"], errors='coerce')
            #ISOLATE PROBOSCIS ACTIVITY
            # Filter rows where:
            #   1. Tube_prob_likelihood or End_prob_likelihood is greater than 0.6
            #   2. End_prob_y is >= 10
            #   3. Tube_prob_y is >= 10
            proboscis_data_to_add = proboscis_data_to_add[
                ((proboscis_data_to_add["Tube_prob_likelihood"] > 0.8) | (proboscis_data_to_add["End_prob_likelihood"] > 0.8)) &
                (proboscis_data_to_add["End_prob_y"] >= 10) &
                (proboscis_data_to_add["Tube_prob_y"] >= 10)
            ].copy()
            #CLASSIFY FRAMES OF PROBOSCIS ACTIVITY
            proboscis_data_to_add['Behaviour'] = proboscis_data_to_add.apply(get_behaviour, axis=1)

            # Write the data to the CSV file
            proboscis_data_to_add.to_csv(
                PATH_EXPORT,
                mode='a',  # Append mode
                index = False,
                header = False # Write header only if file doesn't exist
            )
            file_exists = True  # Ensure subsequent appends don't write headers

            print(f"CSV {counter} processed for proboscis activity and added to feeding file.")

print(f"All {counter} CSV files processed and appended to {PATH_EXPORT}.")

Processing CSV 1 in list: left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 1 processed for proboscis activity and added to feeding file.
Processing CSV 2 in list: left_06-11-2024_16-21-00DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 2 processed for proboscis activity and added to feeding file.
Processing CSV 3 in list: left_06-11-2024_16-21-17DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 3 processed for proboscis activity and added to feeding file.
Processing CSV 4 in list: left_06-11-2024_16-24-45DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 4 processed for proboscis activity and added to feeding file.
Processing CSV 5 in list: left_06-11-2024_16-24-56DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 5 processed for proboscis activity and added to feeding file.
Processing CSV 6 in list: left_06-11-2024_16-25-05DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 6 processed for proboscis activity and added to feeding file.
Processing CSV 7 in li

/tmp/ipython-input-2905771041.py:57: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(file_path)


CSV 912 processed for proboscis activity and added to feeding file.
Processing CSV 913 in list: left_07-11-2024_03-27-59DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 913 processed for proboscis activity and added to feeding file.
Processing CSV 914 in list: left_07-11-2024_03-28-28DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 914 processed for proboscis activity and added to feeding file.
Processing CSV 915 in list: left_07-11-2024_03-28-49DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 915 processed for proboscis activity and added to feeding file.
Processing CSV 916 in list: left_07-11-2024_03-41-11DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 916 processed for proboscis activity and added to feeding file.
Processing CSV 917 in list: left_07-11-2024_03-41-21DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 917 processed for proboscis activity and added to feeding file.
Processing CSV 918 in list: left_07-11-2024_03-54-28DLC_resnet50_Beecam_v4Jun25shuffle1_186000.cs

/tmp/ipython-input-2905771041.py:57: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(file_path)


CSV 931 processed for proboscis activity and added to feeding file.
Processing CSV 932 in list: left_07-11-2024_07-45-57DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 932 processed for proboscis activity and added to feeding file.
Processing CSV 933 in list: right_07-11-2024_07-50-39DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 933 processed for proboscis activity and added to feeding file.
Processing CSV 934 in list: right_07-11-2024_07-51-37DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 934 processed for proboscis activity and added to feeding file.
Processing CSV 935 in list: left_07-11-2024_07-52-25DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 935 processed for proboscis activity and added to feeding file.
Processing CSV 936 in list: right_07-11-2024_07-54-16DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 936 processed for proboscis activity and added to feeding file.
Processing CSV 937 in list: left_07-11-2024_07-54-34DLC_resnet50_Beecam_v4Jun25shuffle1_186000

/tmp/ipython-input-2905771041.py:57: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(file_path)


KeyboardInterrupt: 

---

In [ ]:

# Directory to analyze
directory_list = os.listdir(directory_to_analyse_path)

# Initialize counter
counter = 0

# Loop through directories
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    # Loop through folders
    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, directory, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]

        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(os.path.join(folder_to_analyse_path, 'Videos')) if file.endswith('.csv')]

        for file in csv_files:
            counter += 1
            print(f"Processing CSV {counter} in list: {file}")
            file_path = os.path.join(folder_to_analyse_path, 'Videos', file)

            # Read the CSV file and process
            data_to_add = pd.read_csv(file_path)
            data_to_add = data_to_add.drop([0, 1])  # Remove extraneous rows

            # Rename columns
            data_to_add.columns.values[0] = "Frame"
            data_to_add.columns.values[1] = "L_antenna_x"
            data_to_add.columns.values[2] = "L_antenna_y"
            data_to_add.columns.values[3] = "L_antenna_likelihood"
            data_to_add.columns.values[4] = "R_antenna_x"
            data_to_add.columns.values[5] = "R_antenna_y"
            data_to_add.columns.values[6] = "R_antenna_likelihood"
            data_to_add.columns.values[7] = "L_mandible_x"
            data_to_add.columns.values[8] = "L_mandible_y"
            data_to_add.columns.values[9] = "L_mandible_likelihood"
            data_to_add.columns.values[10] = "R_mandible_x"
            data_to_add.columns.values[11] = "R_mandible_y"
            data_to_add.columns.values[12] = "R_mandible_likelihood"
            data_to_add.columns.values[13] = "Top_prob_x"
            data_to_add.columns.values[14] = "Top_prob_y"
            data_to_add.columns.values[15] = "Top_prob_likelihood"
            data_to_add.columns.values[16] = "Tube_prob_x"
            data_to_add.columns.values[17] = "Tube_prob_y"
            data_to_add.columns.values[18] = "Tube_prob_likelihood"
            data_to_add.columns.values[19] = "End_prob_x"
            data_to_add.columns.values[20] = "End_prob_y"
            data_to_add.columns.values[21] = "End_prob_likelihood"
            # Add metadata columns
            data_to_add["Tube"] = file[:4]
            data_to_add["RPi"] = pi_no
            data_to_add["Video"] = file
            data_to_add["Round"] = Round

            # Write the data to the CSV file
            data_to_add.to_csv(
                output_file,
                mode='a',  # Append mode
                index = False,
                header = False # Write header only if file doesn't exist
            )
            file_exists = True  # Ensure subsequent appends don't write headers

            print(f"CSV {counter} processed and added to file.")

print(f"All {counter} CSV files processed and appended to {output_file}.")


# Old

In [ ]:
#Create a csv file to contain all the data

# Specify the file name
new_dataset = "all_data_Dec_24_xmas.csv" ### EDIT THIS

# Create an empty file
with open(new_dataset, 'w') as file:
    pass  # This creates an empty file

In [ ]:
# Define the output file path
output_file = '/content/drive/My Drive/BeecamData/Other drugs/DLC_output/all_data_Dec24_xmas.csv'

# Check if the file exists
file_exists = os.path.exists(output_file)

# Directory to analyze
directory_list = os.listdir(directory_to_analyse_path)

# Initialize counter
counter = 0

# Loop through directories
for directory in directory_list:
    if directory == 'Died':
        continue
    else:
        Round = directory[6:]
        folder_list = os.listdir(os.path.join(directory_to_analyse_path, directory))

    # Loop through folders
    for folder in folder_list:
        folder_to_analyse_path = os.path.join(directory_to_analyse_path, directory, folder)
        pi_no = folder[7:12]
        if pi_no == 'RPi10':
            start_date = folder[13:23]
            start_time = folder[24:32]
        else:
            start_date = folder[12:22]
            start_time = folder[23:31]

        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(os.path.join(folder_to_analyse_path, 'Videos')) if file.endswith('.csv')]

        for file in csv_files:
            counter += 1
            print(f"Processing CSV {counter} in list: {file}")
            file_path = os.path.join(folder_to_analyse_path, 'Videos', file)

            # Read the CSV file and process
            data_to_add = pd.read_csv(file_path)
            data_to_add = data_to_add.drop([0, 1])  # Remove extraneous rows

            # Rename columns
            data_to_add.columns.values[0] = "Frame"
            data_to_add.columns.values[1] = "L_antenna_x"
            data_to_add.columns.values[2] = "L_antenna_y"
            data_to_add.columns.values[3] = "L_antenna_likelihood"
            data_to_add.columns.values[4] = "R_antenna_x"
            data_to_add.columns.values[5] = "R_antenna_y"
            data_to_add.columns.values[6] = "R_antenna_likelihood"
            data_to_add.columns.values[7] = "L_mandible_x"
            data_to_add.columns.values[8] = "L_mandible_y"
            data_to_add.columns.values[9] = "L_mandible_likelihood"
            data_to_add.columns.values[10] = "R_mandible_x"
            data_to_add.columns.values[11] = "R_mandible_y"
            data_to_add.columns.values[12] = "R_mandible_likelihood"
            data_to_add.columns.values[13] = "Top_prob_x"
            data_to_add.columns.values[14] = "Top_prob_y"
            data_to_add.columns.values[15] = "Top_prob_likelihood"
            data_to_add.columns.values[16] = "Tube_prob_x"
            data_to_add.columns.values[17] = "Tube_prob_y"
            data_to_add.columns.values[18] = "Tube_prob_likelihood"
            data_to_add.columns.values[19] = "End_prob_x"
            data_to_add.columns.values[20] = "End_prob_y"
            data_to_add.columns.values[21] = "End_prob_likelihood"
            # Add metadata columns
            data_to_add["Tube"] = file[:4]
            data_to_add["RPi"] = pi_no
            data_to_add["Video"] = file
            data_to_add["Round"] = Round

            # Write the data to the CSV file
            data_to_add.to_csv(
                output_file,
                mode='a',  # Append mode
                index = False,
                header = False # Write header only if file doesn't exist
            )
            file_exists = True  # Ensure subsequent appends don't write headers

            print(f"CSV {counter} processed and added to file.")

print(f"All {counter} CSV files processed and appended to {output_file}.")


Processing CSV 1 in list: left_06-11-2024_16-20-32DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 1 processed and added to file.
Processing CSV 2 in list: left_06-11-2024_16-21-00DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 2 processed and added to file.
Processing CSV 3 in list: left_06-11-2024_16-21-17DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 3 processed and added to file.
Processing CSV 4 in list: left_06-11-2024_16-24-45DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 4 processed and added to file.
Processing CSV 5 in list: left_06-11-2024_16-24-56DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 5 processed and added to file.
Processing CSV 6 in list: left_06-11-2024_16-25-05DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 6 processed and added to file.
Processing CSV 7 in list: left_06-11-2024_16-26-43DLC_resnet50_Beecam_v4Jun25shuffle1_186000.csv
CSV 7 processed and added to file.
Processing CSV 8 in list: left_06-11-2024_16-26-50DLC_resnet50_Beecam_v4Jun2

KeyboardInterrupt: 

---
